In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')

# visualize results module
from visualize import visualize_svm as Visualize


Autosaving every 180 seconds


<ipython-input-1-0673dc4e64b0>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [8]:
#####################################################
#####################################################
#####################################################
def load_lockout_widths(animal_id):
    
    root_dir = '/media/cat/4TBSSD/yuki/'
    fname = os.path.join(root_dir,
                         animal_id,
                         'event_files.npy')

    fnames = np.load(fname)
    
    return fnames
    
    
def get_session_trials_and_paw_movements(animal_id,
                                         n_sec_lockout):
    
    sample_rate_video =15

    # 
    v = Visualize.Visualize()
    v.main_dir = '/media/cat/4TBSSD/yuki/'
    v.animal_id = animal_id

    # 
    v.session_id = "all"  # use a random date as dummy to load all other sessions
    v.get_fname()


# 
def fit_line(x,y):
    from sklearn import datasets, linear_model
    import scipy
    
    
    #print ("x: ", x)
    #print ("y: ", y)
    corr = scipy.stats.pearsonr(x,y.squeeze())
    #print ("pearson corr: ", corr)
    
    plt.title("pcor: "+str(round(corr[0],5))+ 
              "  pval: "+str(round(corr[1],5)))

    # Create linear regression object
    regr = linear_model.LinearRegression()

    #
    x = x[:,None]
    y = y#[:,None]
    regr.fit(x, y)

    # Make predictions using the testing set
    x_test = x 
    y_test = regr.predict(x_test)

    #

    return x_test, y_test



In [65]:
########################################################
########################################################   
########################################################
import pandas as pd

stroke_sessions = ['sham', 
                   'sham',
                   'IA3_4-11-2016_PM.txt',
                   'IJ1_4-11-2016_PM.txt',
                   'sham',
                   'sham'
                  ]
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
lock_durations = []
for ctr, animal_id in enumerate(animal_ids):
    fnames = load_lockout_widths(animal_id)

    lock_durations.append([])
    for k in range(len(fnames)):
        fname1 = fnames[k]

        try:
            fname1 = fname1.decode("utf-8") 
        except:
            pass
        fname1 = fname1.replace('12TB','4TBSSD').replace('10TB','4TBSSD').replace('/in_vivo/','').replace('tim','')

        
        if os.path.split(fname1)[1]==stroke_sessions[ctr]:
            break
        
        try:
            df = pd.read_csv(fname1, 
                           #header=None, 
                           delimiter=r"\s+")
        except:
            pass

        d1 = df['exp_hold'].to_numpy()

        idx = np.where(d1=='minPos')[0]

        if len(idx)>0:
            d1[idx]=0.0

        d1 = np.float32(d1)

        idx = np.where(d1!=0)[0]
        d1 = d1[idx]
        # print (os.path.split(fname1)[1], round(d1.mean(0),2))
        
        lock_durations[ctr].append(d1.mean(0))
print ("...DONE...")

# 
names = ["M1","M2","M3","M4","M5","M6"]
plt.figure()
for k in range(len(lock_durations)):
    ax=plt.subplot(2,3,k+1)
    plt.title(names[k])
    temp = lock_durations[k]
    print ("temp: ", temp)
    plt.scatter(np.arange(len(temp)), 
                temp)
    plt.ylim(0,1.5)
    plt.ylabel("Duration of lockout (sec)")
    plt.xlabel("Session #")
plt.show()

    

<ipython-input-65-692ab903e9cd>:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  idx = np.where(d1=='minPos')[0]


...DONE...
temp:  [0.10000001, 0.09999999, 0.10000004, 0.1, 0.1, 0.10000001, 0.1, 0.10000001, 0.1, 0.099999994, 0.1, 0.1, 0.099999994, 0.100000024, 0.1, 0.10000003, 0.1, 0.1, 0.099999994, 0.099999994, 0.1, 0.099999994, 0.099999994, 0.099999994, 0.099999994, 0.1, 0.09999999, 0.1, 0.09999999, 0.1, 0.1, 0.10000001, 0.1, 0.099999994, 0.1, 0.1, 0.1, 0.09999999, 0.09999998, 0.1, 0.09999999, 0.1, 0.1, 0.1, 0.09999999, 0.1, 0.10000001, 0.09999999, 0.1, 0.099999994, 0.099999994, 0.1, 0.1, 0.1, 0.099999994, 0.099999994, 0.099999994, 0.09999999, 0.09999999, 0.1, 0.09999999, 0.1, 0.1, 0.099999994, 0.1, 0.1, 0.1, 0.1, 0.09999999, 0.099999994, 0.099999994]
temp:  [0.1, 0.1, 0.1, 0.10000004, 0.100000024, 0.25, 0.25, 0.25, 0.1, 0.1, 0.1, 0.1, 0.1, 0.10000002, 0.100000024, 0.1, 0.10000001, 0.1, 0.099999994, 0.1, 0.09999999, 0.10000001, 0.09999999, 0.099999994, 0.10000001, 0.099999994, 0.1, 0.099999994, 0.10000003, 0.10000002, 0.10000002, 0.10000003, 0.10000004, 0.1, 0.1, 0.10000001, 0.1, 0.1, 0.1, 0.10

In [71]:
import matplotlib.pyplot as plt
import numpy as np

# generate a simple array
x = np.arange(5)
y = np.zeros(5)

#
print ("x: ", x)
print ("y: ", y)

# make a figure
plt.figure()

# scatter plot x vs. y
plt.scatter(x,y)

# show the image
plt.show()

x:  [0 1 2 3 4]
y:  [0. 0. 0. 0. 0.]
